# Image Pre-processing

In this workbook, we will pre-process some images to be used in training a neural network. The images processed here are part of a collection of images of nuclei used in Kaggle's Data Science Bowl 2018 competition. The purpose of the contest is to build a model that can detect the nuclei of the given images. You can learn more about this contest here: https://www.kaggle.com/c/data-science-bowl-2018

First, we need to import some libraries we will be using.

In [1]:
import numpy as np
from PIL import Image
import os
import cv2
import scipy.misc

## Doubling the Data

The first thing we will do is double our dataset. We will do this by flipping the images, and saving a copy of the flipped image, along with the original, each in it's own directory.

Let's set the main directory, then iterate through the directories of images, saving originals and flipped copies of each.

In [2]:
os.chdir('C:/Users/Drew/Desktop/Data_Science_Bowl_18/Image_Prep')
cwd = os.getcwd()
print(cwd)

nuclei_folders = os.listdir(str(cwd+'/stage1_train_fix/'))
os.chdir('stage1_train_fix/')

cwd = os.getcwd()
for i in nuclei_folders:
    new_dir = str(cwd+'/'+i+'_f_v')
    if os.path.isdir(new_dir):
        os.chdir(new_dir)
    else:
        os.mkdir(new_dir)
        os.chdir(new_dir)
    work_dir = os.getcwd()
    
    images_dir = str(work_dir+'/images/')
    if os.path.isdir(images_dir):
        os.chdir(images_dir)
    else:
        os.mkdir(images_dir)
        os.chdir(images_dir)
    image = os.listdir(str(cwd+'/'+i+'/images/'))[0]
    true_img = cv2.imread(str(cwd+'/'+i+'/images/'+image),cv2.IMREAD_UNCHANGED)
    new_img_name = str(image[:-4] + '_f_v.png')
    flip_img = cv2.flip(true_img,0)
    cv2.imwrite(new_img_name,flip_img)
    os.chdir(work_dir)

C:\Users\Drew\Desktop\Data_Science_Bowl_18\Image_Prep


Awesome, we now have double the data.

Let's make sure we've reset our directory.

In [3]:
os.chdir('C:/Users/Drew/Desktop/Data_Science_Bowl_18/Image_Prep')
cwd = os.getcwd()
print(cwd)

C:\Users\Drew\Desktop\Data_Science_Bowl_18\Image_Prep


## Pre-processing the images

Let's now make a few functions we can use to pre-process the images.

This first one will determine if the image has a dark or light background. Most images in the data set have dark backgrounds, and we would like to keep this consistent. Therefore, we would like to invert images that have light backgrounds. This first functions purpose is to return a boolean value True if the image is dark, or False if it is light.

In [4]:
def mostly_black(img_path, black_thresh=50, cutoff=0.5):
    im = Image.open(img_path).convert('L')
    pixels = im.getdata()
    black_thresh = 50
    nblack = 0
    for pixel in pixels:
        if pixel < black_thresh:
            nblack += 1
    n = len(pixels)

    if (nblack / float(n)) > cutoff:
        val = True
    else:
        val = False
    
    return val

Now we can use this boolean value to invert light images with a new function.

In [5]:
def invert_imgs(img_path):
    img = cv2.imread(img_path)
    if mostly_black(img_path):
        return img
    else:
        return cv2.bitwise_not(img)

Before we invert the light images, we would like to strecth the contrast of the light images. Doing this will make it easier for our model to detect the nuclei in the images after they are inverted. The functions below will stretch the contrast of an image, but only if the image is returned with a False value when the mostly_black() function is applied, implying a light background.

In [6]:
# Method to process the red band of the image

def normalizeRed(intensity):
    iI      = intensity    
    minI    = 86
    maxI    = 230
    minO    = 0
    maxO    = 255
    iO      = (iI-minI)*(((maxO-minO)/(maxI-minI))+minO)
    return iO 

# Method to process the green band of the image

def normalizeGreen(intensity):
    iI      = intensity
    minI    = 90
    maxI    = 225
    minO    = 0
    maxO    = 255
    iO      = (iI-minI)*(((maxO-minO)/(maxI-minI))+minO)
    return iO

# Method to process the blue band of the image

def normalizeBlue(intensity):
    iI      = intensity
    minI    = 100
    maxI    = 210
    minO    = 0
    maxO    = 255
    iO      = (iI-minI)*(((maxO-minO)/(maxI-minI))+minO)
    return iO
    

def stretch_contrast(img_path):    
    if mostly_black(img_path):
        img = Image.open(img_path)
        contrasted_img = np.array(img)
    
    else:
        img = Image.open(img_path)
        multiBands = img.split()
        normalizedRedBand   = multiBands[0].point(normalizeRed)
        normalizedGreenBand = multiBands[1].point(normalizeGreen)
        normalizedBlueBand  = multiBands[2].point(normalizeBlue)
        normalized_img = Image.merge("RGB", (normalizedRedBand, normalizedGreenBand, normalizedBlueBand))
        normalized_img.convert('RGB') 
        open_cv_image = np.array(normalized_img) 
        # Convert RGB to BGR
        contrasted_img = open_cv_image[:, :, ::-1].copy()
    
    return contrasted_img

An example of an image with a light background:

https://i.imgur.com/RHO12wu.png

And a dark background:

https://i.imgur.com/wzSDb7v.png

We also want all the images to be the same size. This is important for how the model will read the images. Let's define that function here.

In [7]:
def resize_image(image, min_dim=None, max_dim=None, padding=False):
    """
    Resizes an image keeping the aspect ratio.

    min_dim: if provided, resizes the image such that it's smaller
        dimension == min_dim
    max_dim: if provided, ensures that the image longest side doesn't
        exceed this value.
    padding: If true, pads image with zeros so it's size is max_dim x max_dim

    Returns:
    image: the resized image
    
    """
    # Default window (y1, x1, y2, x2) and default scale == 1.
    h, w = image.shape[:2]
    window = (0, 0, h, w)
    scale = 1

    if min_dim:
        # Scale up but not down
        scale = max(1, min_dim / min(h, w))
    # Does it exceed max dim?
    if max_dim:
        image_max = max(h, w)
        if round(image_max * scale) > max_dim:
            scale = max_dim / image_max
    # Resize image and mask
    if scale != 1:
        image = scipy.misc.imresize(
            image, (round(h * scale), round(w * scale)))
    # Need padding?
    if padding:
        # Get new height and width
        h, w = image.shape[:2]
        top_pad = (max_dim - h) // 2
        bottom_pad = max_dim - h - top_pad
        left_pad = (max_dim - w) // 2
        right_pad = max_dim - w - left_pad
        padding = [(top_pad, bottom_pad), (left_pad, right_pad), (0, 0)]
        image = np.pad(image, padding, mode='constant', constant_values=0)
        window = (top_pad, left_pad, h + top_pad, w + left_pad)
    return image

We can now iterate through the images and apply these functions. The resulting images will be stored in a new directory, titled 'Images'.

In [8]:
nuclei_folders = os.listdir(str(cwd+'/stage1_train_fix/'))
os.chdir('stage1_train_fix/')

cwd = os.getcwd()

for i in nuclei_folders:
    work_dir = os.getcwd()
    images_dir = str(work_dir+'/images/')
    if os.path.isdir(images_dir):
        os.chdir(images_dir)
    else:
        os.mkdir(images_dir)
        os.chdir(images_dir)
    image = os.listdir(str(cwd+'/'+i+'/images/'))[0]
    
    new_img_name = str(image[:-4] + 'new.png')
    new_img = stretch_contrast(str(cwd+'/'+i+'/images/'+image))
    cv2.imwrite(new_img_name,new_img)
    new_img = invert_imgs(new_img_name)
    new_img = resize_image(new_img, 800, 1042, padding = True)
    cv2.imwrite(new_img_name,new_img)

    os.chdir(work_dir)

After pre-processing, the example light image now appears as this:

https://imgur.com/nVLREMY

while the dark image remains the same, but resized:

https://imgur.com/Lr5KmLF